In [ ]:
#import library
import cv2
from torchvision import transforms, models, datasets
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
#build cascade obj
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
#define vgg16 model
def get_model():
    model = models.vgg19(pretrained = True)
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.Sequential(
        nn.Conv2d(512,512, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten()
    )
    class ageGenderClassifier(nn.Module):
        def __init__(self):
            super(ageGenderClassifier, self).__init__()
            self.intermediate = nn.Sequential(
                nn.Linear(2048,512),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(512,128),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(128,64),
                nn.ReLU(),
            )
            self.age_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
            self.gender_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        def forward(self, x):
            x = self.intermediate(x)
            age = self.age_classifier(x)
            gender = self.gender_classifier(x)
            return gender, age

    model.classifier = ageGenderClassifier()

    return model.to(device)


In [ ]:
def model_prediction(pre_img, model):

    face = pre_img.to(device).float()
    gender, age = model(face)

    pred_gender = gender.to('cpu').detach().numpy()
    pred_age = age.to('cpu').detach().numpy()

    return pred_gender, pred_age

In [ ]:

#define detection function
def detection(gray):

    face = face_cascade.detectMultiScale(gray, 1.3, 5)   

    return face


In [ ]:
#preprocess function
def preprocess_image(face, frame):
    for x, y, w, h in face:
        frame = np.array(frame)
        crop_frame = frame[y:y+h, x:x+w]

        crop_frame = cv2.resize(crop_frame, (224, 224))
        crop_frame = torch.tensor(crop_frame).permute(2,0,1)
        crop_frame = normalize(crop_frame/255.)

        return crop_frame[None]

In [ ]:
model= get_model()
model.load_state_dict(torch.load("vgg19", map_location=torch.device('cpu')))

In [ ]:
#Capture video from webcam

cap_vid = cv2.VideoCapture(0)

while True:

    _, frame = cap_vid.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    detect_face = detection(gray)

    pre_img = preprocess_image(detect_face, frame)
    imgee = pre_img
    gender, age = model_prediction(pre_img, model)
    
    gender = np.where(gender[0][0]<0.5,'Male','Female')
    age = int(age[0][0]*116)
    for x, y, w, h in detect_face:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (100, 50, 200), 3)

        cv2.putText(frame, f'gender:{gender}, age: {age}', (x, w+h), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('detected_face',frame)
    if cv2.waitKey(0) & 0xff == ord("q"):
        break

In [18]:
cap_vid.release()
cv2.destroyAllWindows()